In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)  
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv("E:\\WorkSpace\\Complete_MLOps_Bootcamp_With_10_End_To_End_ML_Projects\\7_credit_fraud_detection\\data\\default_of_credit_card_clients.csv")
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
df.index = df['ID']
df = df.drop(['ID'], axis='columns')
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [6]:
for key, value in df['default payment next month'].value_counts().items():
        print(f"Percentage value for {key} =",  value/df.shape[0] * 100)

Percentage value for 0 = 77.88000000000001
Percentage value for 1 = 22.12


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [8]:
import pandas as pd
pd.set_option('display.max_columns', None)  
import numpy as np
import mlflow
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

def oversample_training_data(X_train, y_train, random_state=42):
    """
    Oversample the minority class in training data using SMOTE.
    
    Parameters:
    -----------
    X_train : array-like
        Training features
    y_train : array-like
        Training labels
    random_state : int, optional (default=42)
        Random state for reproducibility
        
    Returns:
    --------
    X_train_resampled : array-like
        Resampled training features
    y_train_resampled : array-like
        Resampled training labels
    """
    # Initialize SMOTE
    smote = SMOTE(random_state=random_state)
    
    # Fit and transform the training data
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    # Print class distribution before and after resampling
    print("Class distribution before resampling:")
    print(pd.Series(y_train).value_counts(normalize=True))
    print("\nClass distribution after resampling:")
    print(pd.Series(y_train_resampled).value_counts(normalize=True))
    
    return X_train_resampled, y_train_resampled


X, y = df.drop(['default payment next month'], axis='columns'), df['default payment next month']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_sampled, y_train_sampled = oversample_training_data(X_train, y_train, random_state=42)
X_train_sampled.shape, y_train_sampled.shape, X_train.shape, y_train.shape

Class distribution before resampling:
default payment next month
0    0.778208
1    0.221792
Name: proportion, dtype: float64

Class distribution after resampling:
default payment next month
0    0.5
1    0.5
Name: proportion, dtype: float64


((37354, 23), (37354,), (24000, 23), (24000,))

In [9]:
# generate a random forest parameter dictionary to find best performing model
rf_params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# generate Logistic Regression parameter dictionary to find best performing model
lr_params = {
    'C': [0.001, 0.1, 1, 10],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'solver': ['lbfgs', 'liblinear']
}

# grid search for random forest
rf_grid_searh = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42), 
    param_grid=rf_params, 
    scoring='f1', 
    cv=5, 
    verbose=1, 
    n_jobs=-1
)

# grid search for logistic regression
lr_grid_search = GridSearchCV(
estimator=LogisticRegression(),
    param_grid=lr_params,
    scoring='f1',
    cv=5,
    n_jobs=-1,
)


In [10]:
print("Searching best combination for Logistic Regression model...")
lr_grid_search.fit(X_train, y_train)
print("Searching for Logistic Regression done ...")

Searching best combination for Logistic Regression model...


e:\WorkSpace\Complete_MLOps_Bootcamp_With_10_End_To_End_ML_Projects\7_credit_fraud_detection\env\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "e:\WorkSpace\Complete_MLOps_Bootcamp_With_10_End_To_End_ML_Projects\7_credit_fraud_detection\env\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "e:\WorkSpace\Complete_MLOps_Bootcamp_With_10_End_To_End_ML_Projects\7_credit_fraud_detection\env\Lib\site-packages\sklearn\base.py", line 1389, in wrapper
  

Searching for Logistic Regression done ...


In [11]:
print("Searching best combination for Random Forest model...")
rf_grid_searh.fit(X_train, y_train)
print("Searching for Random Forest done ...")

Searching best combination for Random Forest model...
Fitting 5 folds for each of 54 candidates, totalling 270 fits


KeyboardInterrupt: 

In [11]:
lr_best_params = lr_grid_search.best_params_
rf_best_params = rf_grid_searh.best_params_
lr_best_params, rf_best_params

({'C': 1, 'penalty': 'l1', 'solver': 'liblinear'},
 {'max_depth': 10,
  'min_samples_leaf': 4,
  'min_samples_split': 10,
  'n_estimators': 100})

In [12]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return accuracy, precision, recall, f1  

In [14]:
# Now we can use mlflow to log these parameters
# mlflow.set_tracking_uri("http://localhost:5000")  # Set your MLflow tracking URI
import os
mlflow.set_tracking_uri("https://dagshub.com/vijaytakbhate2002/credit_fraud_detection_project_with_mlflow_dagshub_dvc.mlflow/")  # Set your experiment name
os.environ["MLFLOW_TRACKING_USERNAME"] = "vijaytakbhate20@gmail.com" # Set your DagsHub username
os.environ["MLFLOW_TRACKING_PASSWORD"] = ""  # Set your DagsHub credentials

mlflow.set_experiment("Credit Card Fraud Detection")
with mlflow.start_run(run_name="Logistic Regression Model") as run:
    mlflow.log_params(lr_best_params)


    best_model = lr_grid_search.best_estimator_
    accuracy, precision, recall, f1 = evaluate_model(best_model, X_test, y_test)


    mlflow.log_metric("accuracy", accuracy) 
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Log feature importances if applicable
    feature_importances = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.coef_[0]
    }).sort_values(by='importance', ascending=False)
    
    feature_importances.to_csv("feature_importances.csv", index=False)
    mlflow.log_artifact("feature_importances.csv", artifact_path="feature_importances.csv")


    mlflow.sklearn.log_model(lr_grid_search.best_estimator_, "model")

    print("Logged Logistic Regression model with best parameters to MLflow")


🏃 View run Logistic Regression Model at: https://dagshub.com/vijaytakbhate2002/credit_fraud_detection_project_with_mlflow_dagshub_dvc.mlflow/#/experiments/0/runs/296f9e015c524b9cb24f7aed45bcd7c8
🧪 View experiment at: https://dagshub.com/vijaytakbhate2002/credit_fraud_detection_project_with_mlflow_dagshub_dvc.mlflow/#/experiments/0


NameError: name 'lr_best_params' is not defined

In [13]:

with mlflow.start_run(run_name="Random Forest Model") as run:
    mlflow.log_params(rf_best_params)
    best_rf_model = rf_grid_searh.best_estimator_
    accuracy, precision, recall, f1 = evaluate_model(best_rf_model, X_test, y_test)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.sklearn.log_model(rf_grid_searh.best_estimator_, "model")
    print("Logged Random Forest model with best parameters to MLflow")

MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 403 != 200. Response body: ''